本文内容主要来自机器之心https://www.jiqizhixin.com/articles/2017-09-08

非凸游湖中一个核心问题是如何避开鞍点。尽管进来的研究已经表明梯度下降GD通常可以在迭代了极其多步后最终避开鞍点，但仍然还有一个需要解决的关键问题：效率。GD能快速避开鞍点吗？还是会在鞍点附近显著减速？避开鞍点的速率又会随着参数个数（维度）的变化而怎样变化？

## 扰动梯度下降
在经典梯度下降邻域，给定一个函数f，我们的目标是通过在负梯度的方向上移动来找到该函数的最小化。在优化函数是凸函数的情况下，GD已经有了非常好的理论解释；但当优化函数非凸时，已有的研究要少得多。我们仅知道在非凸优化中，GD可以快速收敛到驻点（$\triangledown f(x)=0$的点），这些驻点可能是局部最小点，但也可能是毫无用处的局部最大点或鞍点。<br>
显然如果GD是从一个鞍点开始的，那么它就根本就不会离开那里，即使那可能是一个局部最大值点；因此为了使GD能避开次优的驻点（例如鞍点），我们必须对GD稍微做修改，加入一定程度的随机性。目前的文献已经研究过两种加入随机性的简单方法：<br>
1. 间歇的扰动： 论文“escaping from saddle points -- online Stochastic Gradient for Tensor Decomposition”考虑了在GD中偶尔加入随机扰动的算法，并且第一次为GD避开鞍点提供了多项式时间保证。<br>
2. 随机初始化：论文《Gradient Descent Converges to Minimizers》表明仅需要随机初始化，就可以确保GD能渐进地避开鞍点。<br>

渐进性的，甚至多项式时的结果一般来说都很重要，但他们却不能很好的解释为什么很多基于梯度的算法在实际的非凸问题上效果非常好。而且他们也无法保证当用户观察到一段相对平坦的学习曲线时，究竟是在鞍点附近还是已经收敛到局部最小值了。最后，他们更无法保证GD在非凸优化中能像在凸优化中那样告诉解决高维问题。<br>
对于避开鞍点，一种合理方法是使用二阶(基于Hessian的)算法。尽管这些算法在每步跟新中计算成本比GD(远远)更高，而且实现起来也更加复杂，但Hessian确实能提供关于鞍点的几何信息，从而在较少的步数内快速避开鞍点。因此，在学界中已经有相当一部分工作给出了基于Hessian的算法的较好理论解释，并且也得到了正面的结论。<br>
## GD也可以快速避开鞍点吗？还是说hessian对快速避开鞍点是必须的？
如果仅限于考虑上文提及的“随机初始化”策略，对于这种策略第一个问题的答案是否定的。事实上，这种方法可以被证明一般来说效率很低，在最坏的案例中避开鞍点甚至需要指数级的大量时间<br>
但如果我们考虑“间歇的扰动”策略，我们却能得到一个相当不同而且正面的结果，这或多或少有点让人惊讶。为了称述这个结果，让我们先澄清一下我们要使用的算法：<br>
Perturbed gradient descent (PGD)<br>
$\ \ \ \ $    for t = 1, 2, ... do<br>
$\ \ \ \ \ \ $        $x_t <- x_{t-1} - \eta\Delta f(x_{t-1})$<br>
$\ \ \ \ \ \ $        if perturbation condition holds then <br>
$\ \ \ \ \ \ \ \ \ \ $            $x_t <- x_t + \xi_t$<br>
其中$\xi$是一个以0位中心，半径适当小的球上均匀采样的；而且当梯度适当小的时候，这个扰动会被加入到迭代中，这些特定的超参数选择是为了方便分析，我们相信均匀的噪声并不是必须的，值在梯度很小时才加入噪声也不是必要的。
## 严格鞍点和二阶驻点
后文中讨论的“鞍点”同时指代了一般定义中的鞍点，和局部最大点。这些点是在至少一个方向上为局部最大值的驻点。鞍点和局部最小值可以根据Hessian的最小特征值分类：<br>
- 局部最小：> 0
- 局部最小或鞍点： = 0
- 鞍点： < 0

更进一步的说，我们将最后一类鞍点称为严格鞍点(strict saddle points)。
![saddle point](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/saddle%20point.png)
尽管非严格鞍点在谷底可能是平坦的，但严格鞍点要求至少有一个方向的曲率是严格为负的。这样一个方向的存在让基于梯度的算法又了避开鞍点的可能。一般而言，区分局部最小值和非严格鞍点是NP-hard的，因此，我们以及之前的研究者关注的都是避开严格鞍点。<br>
在形式上，我们做了两个关于光滑度的假设，如下：
- f is l-gradient-Lipschitz, i.e.: $\vee x_1,x_2, |\triangledown f(x_1)-\triangledown f(x_2)|<= l|x_1-x_2|$
- f is $\rho$-Hessian_lipschitz, i.e.: $\vee x_1,x_2,|\triangledown^2f(x_1)-\triangledown^2f(x_2)|<= \rho|x_1-x_2|$

经典理论研究收敛到一阶驻点$\triangledown f(x)=0$的速度，是通过研究找到$\varepsilon-一阶驻点(|\triangledown f(x)|<=\varepsilon)$所需的步数。与此类似，我们研究避免严格鞍点的熟读，也就是收敛到二阶驻点$\triangledown f(x)=0, \lambda_{min}(\triangledown^2f(x))>=0$的速度，通过研究找到它的$\varepsilon-$弱化版本。<br>
- Definition: A point x is an $\varepsilon$-second-order stationary point if:
$|\triangledown f(x)|<= \varepsilon\ and\ \lambda_{min}(\triangledown^2f(x))>=-\sqrt{\rho\varepsilon}$
在这个定义中，$\rho$是前面引入的Hessian Lip schitz常数。

## 应用
在实际应用中，一大类非凸问题都可以被证明具有以下性质：所有的鞍点都是严格鞍点。这类问题包括，且不仅限于主成分分析、典型相关分析、正交张量分解、相位回复、字典学习、矩阵感知（matrix sensing）、矩阵补全和其他非凸低秩问题。<br>
此外，在所有上述非凸问题中，也可以证明：所有局部最小值都是全局最小值。因此，在这些情况下，任何可以寻找$\varepsilon$-二阶驻点的通用有效的算法都可以找到全局最小值，从而快速解决这些非凸问题。

## 只需要极小二外开销即可避免鞍点
对于经典理论研究的一阶驻点，我们知道GD在理论上有非常好的性质：<br>
- Theorem(Nesterov 1998): if assumption 1 holds, then GD, with $\eta$ = 1/l, finds an $\varepsilon$-first-order stationary point in $2l(f(x_0)-f^*)/\varepsilon^2$ iterations.

在这个定理中，$x_0$是初始点，$f^*$是全局最小的函数值。该定理说对于任意梯度-Lipschitz函数，GD都可以在$O(1/\varepsilon^2)$步内找到一个驻点，且所需要步数对维度d没有显示的依赖；当然每步中进行梯度计算的时间是O(d)，因此GD的整体运行时间会为O(d)的阶。对于深度学习等现代高维非凸问题而言，O(d)这样的线性运算时间是GD非常重要的优势（相较于二阶算法而言）。<br>
对应地，我们现在同样希望能快速收敛到二阶驻点。对于这个问题，我们能期望得到的最好的结果是什么样的呢：
- 维度无关的迭代次数
- $O(1/\varepsilon^2)$收敛率
- 对l和$(f(x_0)-f^*)$的依赖于Nesterov 1998中的结果完全相同。

相当让人惊讶的是，对于这三个问题，我们给出的答案都是肯定的（额外开销只有一些很小的对数因子）

未完。。。估计不会续了

# 还存在的问题
1. 使用二阶(基于Hessian的)算法，表示并不了解，都是什么根什么啊？？？
2. 光滑度假设表示不懂？？？
3. 应用中得出的结论表示一头雾水

# 补充知识
Hessian矩阵，黑塞矩阵（Hessian Matrix），又译作海森矩阵、海瑟矩阵、海塞矩阵等，是一个多元函数的二阶偏导数构成的方阵，描述了函数的局部曲率。Hessian矩阵的特征值就是形容其在该点附近特征向量方向的凹凸性，特征值越大，凸性越强。你可以把函数想想成一个小山坡，陡的那面是特征值大的方向，平缓的是特征值小的方向。而凸性和优化方法的收敛速度有关，比如梯度下降。如果正定Hessian矩阵的特征值都差不多，那么梯度下降的收敛速度越快，反之如果其特征值相差很大，那么收敛速度越慢。简单粗暴点的说，Hessian矩阵的特征值控制了更新步长。https://www.zhihu.com/question/24623031